## Importacion de las librerias necesarias

In [2]:
import numpy as np
import pandas as pd
import pyarrow
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

## Carga del archivo parquet del data_movies

In [4]:
data_movies = pd.read_parquet("../Source/data_movies.parquet", engine="pyarrow")

## Revisando algunos datos, como cantidad de ceros, NaN, etc.

In [6]:
data_movies

,budget,id,original_language,overview,popularity,release_date,revenue,status,title,vote_average,vote_count,release_year,return,genre_name
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,Released,Toy Story,7.7,5415.0,1970-01-01 00:00:00.000001995,12.451801,Animation
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,Released,Toy Story,7.7,5415.0,1970-01-01 00:00:00.000001995,12.451801,Comedy
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,Released,Toy Story,7.7,5415.0,1970-01-01 00:00:00.000001995,12.451801,Family
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,Released,Jumanji,6.9,2413.0,1970-01-01 00:00:00.000001995,4.043035,Adventure
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,Released,Jumanji,6.9,2413.0,1970-01-01 00:00:00.000001995,4.043035,Fantasy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45463,0.0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,Released,Betrayal,3.8,6.0,1970-01-01 00:00:00.000002003,0.000000,Action
45463,0.0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,Released,Betrayal,3.8,6.0,1970-01-01 00:00:00.000002003,0.000000,Drama
45463,0.0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,Released,Betrayal,3.8,6.0,1970-01-01 00:00:00.000002003,0.000000,Thriller
45464,0.0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,Released,Satan Triumphant,0.0,0.0,1970-01-01 00:00:00.000001917,0.000000,None


In [5]:
data_movies.isnull().sum()

budget                 0
id                     0
original_language     17
overview               0
popularity             0
release_date           0
revenue                0
status               124
title                  0
vote_average           0
vote_count             0
release_year           0
return                 0
genre_name             0
dtype: int64

* ### Si tomamos la cantidad total del datasets (93420 filas) vemos que la cantidad de valores nulos no es significativa.

* ### Lo que mas me preocupa es la cantidad de 0 (Ceros)

In [7]:
(data_movies == 0).sum()

budget               71512
id                       0
original_language        0
overview                 0
popularity               0
release_date             0
revenue              74903
status                   0
title                    0
vote_average          4436
vote_count            4261
release_year             0
return               79400
genre_name               0
dtype: int64

### La columna return tiene una cantidad significativa de 0 (Ceros). Esto se debe tambien a como esta calculada. Ya que en la columna de sus operadores tambien hay muchos 0 (Ceros)